### Step 1: Extract the texts from Digital Periegisis

Inputs:

- `GALAXY_INPUTS["reportStartIndex"]`
- `GALAXY_INPUTS["reportEndIndex"]`

Outputs:

- 'outputs/1_pausanias.jsonl' - A JSONL Dataset with clean text parsed from the web source

In [ ]:
# Only run on Galaxy
!pip install tqdm requests beautifulsoup4 srsly

In [ ]:
import tqdm
import requests
import bs4
import re
import time
import srsly

In [ ]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"}

final_list = []

for number in tqdm.tqdm(range(GALAXY_INPUTS["reportStartIndex"], GALAXY_INPUTS["reportEndIndex"])):
    response = requests.get(f"https://www.periegesis.org/en/reports.php?reportid={number}", headers=headers)
    soup = bs4.BeautifulSoup(response.content, "html.parser")

    x = soup.find("div", class_="text_max_width")

    for tag in soup.find_all(["b", "strong"]):
            tag.decompose()

    chapter = soup.find("h2", class_="head").text
    book = soup.find("h5", class_="head align_center").text

    for i in x.find_all("p"):
        metadata = []
        clean_text = re.split(r"BOOK\s+\d+", i.text)[-1].strip()
        for pl in i.find_all("pl"):
            if pl.get("id").startswith("Q"):
                metadata.append({"chapter":int(chapter.split()[-1]),
                                "book":int(book.split()[-1])})

        final_list.append({"text":clean_text,"metadata":metadata})

    time.sleep(1)

srsly.write_jsonl(f"outputs/1_pausanias.jsonl", final_list)